In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from IPython.core.display_functions import display
from biogeme.expressions import Beta, Variable
from biogeme.models import loglogit
from biogeme.segmentation import DiscreteSegmentationTuple, segmented_beta

In [22]:
df = pd.read_csv("lpmc01.dat", sep = '\t')
database = db.Database('lpmc01', df)


## Variable Definition

In [34]:
trip_id = Variable('trip_id')
household_id = Variable('household_id')
person_n = Variable('person_n')
trip_n = Variable('trip_n')
travel_mode = Variable('travel_mode')
purpose = Variable('purpose')
fueltype = Variable('fueltype')
faretype = Variable('faretype')
bus_scale = Variable('bus_scale')
survey_year = Variable('survey_year')
travel_year = Variable('travel_year')
travel_month = Variable('travel_month')
travel_date = Variable('travel_date')
day_of_week = Variable('day_of_week')
start_time = Variable('start_time')
age = Variable('age')
female = Variable('female')
driving_license = Variable('driving_license')
car_ownership = Variable('car_ownership')
distance = Variable('distance')
dur_walking = Variable('dur_walking')
dur_cycling = Variable('dur_cycling')
dur_pt_access = Variable('dur_pt_access') # Predicted total access and egress time for public transport route in hours
dur_pt_rail = Variable('dur_pt_rail')
dur_pt_bus = Variable('dur_pt_bus')
dur_pt_int = Variable('dur_pt_int') # Time taken (hrs) at each interchange point
pt_interchanges = Variable('pt_interchanges')   # Number of interchange points in public transport route
dur_driving = Variable('dur_driving')
cost_transit = Variable('cost_transit')
cost_driving_fuel = Variable('cost_driving_fuel')   # Estimated fuel cost of driving route in GBP
cost_driving_ccharge = Variable('cost_driving_ccharge')  # Estimated congestion charge cost of driving route in GBP
driving_traffic_percent = Variable('driving_traffic_percent')

In [44]:
# Define driving cost
cost_driving = cost_driving_ccharge + cost_driving_fuel

# Define time taken by each mode of transport
time_pt = dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int


time_cycling = dur_cycling
time_walking = dur_walking
time_driving = dur_driving

# Define transport availability
car_av = (driving_license != 0) * (car_ownership != 0)
pt_av = 1
cycle_av = 1
walking_av =1

# Model 0

In [25]:
# Define betas
Beta_time = Beta('Beta_time', 0, None, None, 0)
Beta_cost = Beta('Beta_cost', 0, None, None, 0)

# Define ASCs

ASC_car = Beta('ASC_car', 0, None, None, 0)
ASC_pt = Beta('ASC_pt', 0, None, None, 0)
ASC_cycling = Beta('ASC_cycling', 0, None, None, 0) 


In [26]:
# Define utility functions

V0_car = ASC_car + Beta_time * time_driving + Beta_cost * cost_driving
V0_pt = ASC_pt + Beta_time * time_pt + Beta_cost * cost_transit
V0_cycling = ASC_cycling + Beta_time * time_cycling 
V0_walking = Beta_time * time_walking

In [58]:
V0 = {1: V0_walking, 2: V0_cycling, 3: V0_pt, 4: V0_car}


In [62]:
av = {1: walking_av, 2: cycle_av, 3: pt_av, 4: car_av}


In [66]:
logprob = loglogit(V0, av,  travel_mode)


In [67]:
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'Model_0'

The chosen alternative [travel_mode] is not available for the following observations (rownumber[choice]): 28[4.0]-36[4.0]-44[4.0]-54[4.0]-55[4.0]-59[4.0]-64[4.0]-88[4.0]-106[4.0]-110[4.0]-112[4.0]-118[4.0]-...


In [68]:
results = biogeme.estimate()
print(results.get_estimated_parameters())

                Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_car     -1.287994      0.090425   -14.243844           0.0
ASC_cycling -3.778167      0.098453   -38.375443           0.0
ASC_pt      -0.525657      0.053008    -9.916589           0.0
Beta_cost   -0.190833      0.021385    -8.923815           0.0
Beta_time   -5.598729      0.180252   -31.060488           0.0
